In [1]:
from __future__ import division
import os
from PIL import Image
import numpy as np
import matplotlib.image as mpimg
from skimage import color
import matplotlib.pyplot as plt 
from pylab import imread,subplot,imshow,title,gray,figure,show,NullLocator
import scipy.misc
import math

In [2]:
lr = 1e-4
reg = 500
num_iters=10000
const2 = 1e50

In [3]:
input_file = open("sample_train_dataset.txt", "r")
a = []
a_name = []
name_classes = []
for file in input_file:
    arr = np.array(Image.open(file.split()[0]).convert("L").resize((32,32))).flatten()
    a.append(arr)
    a_name.append(str(file.split()[1]))
    name_classes.append(str(file.split()[1]))
a_name = np.array(a_name)
unique_classes = list(set(a_name))


In [4]:
x_files = open("sample_test_dataset.txt", "r")
file_name = []
res = []
for file in x_files:
    file = file.split('\n')
    f = Image.open(str(file[0]))
    f = f.convert("L")
    f = f.resize((32,32))
    file_name.append(str(file[0]))
    arr = np.asarray(f)
    arr = arr.reshape(-1)
    res.append(arr)
    f.close



In [5]:
a = np.array(a)
mean = np.mean(a,axis=0)
mean1 = a - mean
v, eigenvalues , eigenvectors = np.linalg.svd(mean1)
eigenvectors = eigenvectors.T
coefficiets = np.matmul(mean1 , eigenvectors) 
eigenvectors = eigenvectors.T
arr1 = np.array(coefficiets[:,:32])
b = np.ones((len(arr1),1))
arr1 = np.hstack((arr1,b))
new_img = np.array(arr1)
eigenvectors = eigenvectors[0:32,:]
eigenvectors = eigenvectors.T

In [6]:
const =  0.001

In [7]:
assign_int = dict()
assign_name = dict()
rang = len(unique_classes)
for i in range(rang):
    assign_int[i] = unique_classes[i]
    assign_name[unique_classes[i]] = i
    
rang  = len(a_name)
for i in range(rang):
    a_name[i] = assign_name[a_name[i]]
a_name = a_name.astype(int)


In [8]:
num_classes = len(set(a_name))
newt = new_img.T
X = np.array(newt)
D = len(X)
W = np.random.randn(num_classes, D) *const
Y = np.array(a_name)

In [13]:
len(W)

8

In [14]:
losses_history = []
for i in range(num_iters):
    loss = 0 
    grad = np.zeros_like(W)
    dim  = X.shape[0]
    num_train = X.shape[1]
    scores = W.dot(X)
    scores -= np.max(scores)
    scores_exp = np.exp(scores)
    scores_exp = scores_exp*const2
    rang = range(num_train)
    correct_scores_exp = scores_exp[Y, rang]
    scores_exp_sum = np.sum(scores_exp, axis=0)
    frac = correct_scores_exp / scores_exp_sum
    loss = -np.sum(np.log(frac))
    loss /= num_train
    scores_exp_normalized = scores_exp / scores_exp_sum
    rang = range(num_train)
    scores_exp_normalized[Y, rang] -= 1
    XT = X.T
    grad = scores_exp_normalized.dot(X.T)
    grad /= num_train
    losses_history.append(loss)
    prod = lr * grad
    W -= prod
    if(loss<1):
        break;
    

In [10]:
res = res - mean
coefficiets = np.matmul(res , eigenvectors)
len1 = len(res)
b = np.ones((len1,1))
res = np.hstack((coefficiets,b))
array1 = np.array(res).T
shap = array1.shape[1]
pred_ys = np.zeros(shap)
scores = W.dot(array1)
pred_ys = np.argmax(scores, axis=0)
pred = pred_ys

In [14]:
print((pred))

[4 4 6 0 7 1 5 4 7 4 2 7 1 4 6 0 7 4 6 6 6 5 3 3 6 6 0 1 5 5 6 1 4 7 7 3 7
 0 4 7 6 4 0 1 3 3 3 2 3 2 0 1 1 3 5 4 5 3 3 0 1 1 0 4 2 5 0 7 5 4 2 2 3 6
 5 1 2 0 1 5 5 1 5 6 0 5 2 6 3 2 2 4 5 7 5 0 1 6 2 7 0 2 1 1 4 5 7 2 3 5 6
 1 2 3 5 0 3 5 2 4]


In [12]:
acc = 0
for i in range(len(pred)):
     if (assign_int[pred[i]] == file_name[i][12]):
            acc=acc+1
print(acc)

73
